In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Human Numbers 

In [2]:
from fastai.text import *

In [3]:
bs = 64

## Data 

In [4]:
path = untar_data(URLs.HUMAN_NUMBERS, "human_numbers", "/home/ricky/Desktop/repos/fastai_course/data")

In [5]:
path.ls()

[PosixPath('/home/ricky/Desktop/repos/fastai_course/data/human_numbers/train.txt'),
 PosixPath('/home/ricky/Desktop/repos/fastai_course/data/human_numbers/valid.txt')]

In [6]:
# concate all words seperated by commas then place in a list
def readnums(d): 
    return [', '.join(o.strip() for o in open(path/d).readlines())]

In [7]:
!head -n 25 data/human_numbers/train.txt

one 
two 
three 
four 
five 
six 
seven 
eight 
nine 
ten 
eleven 
twelve 
thirteen 
fourteen 
fifteen 
sixteen 
seventeen 
eighteen 
nineteen 
twenty 
twenty one 
twenty two 
twenty three 
twenty four 
twenty five 


In [52]:
!tail -n 25 data/human_numbers/train.txt

seven thousand nine hundred seventy five 
seven thousand nine hundred seventy six 
seven thousand nine hundred seventy seven 
seven thousand nine hundred seventy eight 
seven thousand nine hundred seventy nine 
seven thousand nine hundred eighty 
seven thousand nine hundred eighty one 
seven thousand nine hundred eighty two 
seven thousand nine hundred eighty three 
seven thousand nine hundred eighty four 
seven thousand nine hundred eighty five 
seven thousand nine hundred eighty six 
seven thousand nine hundred eighty seven 
seven thousand nine hundred eighty eight 
seven thousand nine hundred eighty nine 
seven thousand nine hundred ninety 
seven thousand nine hundred ninety one 
seven thousand nine hundred ninety two 
seven thousand nine hundred ninety three 
seven thousand nine hundred ninety four 
seven thousand nine hundred ninety five 
seven thousand nine hundred ninety six 
seven thousand nine hundred ninety seven 
seven thousand nine hundred ninety eight 
seven thousand nine 

In [53]:
!head -n 25 data/human_numbers/valid.txt

eight thousand one 
eight thousand two 
eight thousand three 
eight thousand four 
eight thousand five 
eight thousand six 
eight thousand seven 
eight thousand eight 
eight thousand nine 
eight thousand ten 
eight thousand eleven 
eight thousand twelve 
eight thousand thirteen 
eight thousand fourteen 
eight thousand fifteen 
eight thousand sixteen 
eight thousand seventeen 
eight thousand eighteen 
eight thousand nineteen 
eight thousand twenty 
eight thousand twenty one 
eight thousand twenty two 
eight thousand twenty three 
eight thousand twenty four 
eight thousand twenty five 


In [54]:
!tail -n 25 data/human_numbers/valid.txt

nine thousand nine hundred seventy five 
nine thousand nine hundred seventy six 
nine thousand nine hundred seventy seven 
nine thousand nine hundred seventy eight 
nine thousand nine hundred seventy nine 
nine thousand nine hundred eighty 
nine thousand nine hundred eighty one 
nine thousand nine hundred eighty two 
nine thousand nine hundred eighty three 
nine thousand nine hundred eighty four 
nine thousand nine hundred eighty five 
nine thousand nine hundred eighty six 
nine thousand nine hundred eighty seven 
nine thousand nine hundred eighty eight 
nine thousand nine hundred eighty nine 
nine thousand nine hundred ninety 
nine thousand nine hundred ninety one 
nine thousand nine hundred ninety two 
nine thousand nine hundred ninety three 
nine thousand nine hundred ninety four 
nine thousand nine hundred ninety five 
nine thousand nine hundred ninety six 
nine thousand nine hundred ninety seven 
nine thousand nine hundred ninety eight 
nine thousand nine hundred ninety nine 


In [8]:
for line in open(path/"train.txt").readlines()[:25]:
    print(line.strip())

one
two
three
four
five
six
seven
eight
nine
ten
eleven
twelve
thirteen
fourteen
fifteen
sixteen
seventeen
eighteen
nineteen
twenty
twenty one
twenty two
twenty three
twenty four
twenty five


In [9]:
train_txt = readnums("train.txt")

In [10]:
len(train_txt)

1

In [11]:
train_txt[0][:50]

'one, two, three, four, five, six, seven, eight, ni'

In [14]:
train_txt[0][-50:]

'ety eight, seven thousand nine hundred ninety nine'

In [15]:
valid_txt = readnums("valid.txt")

In [18]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs)

In [25]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [26]:
len(data.valid_ds[0][0].data)

13017

In [27]:
data.bptt, len(data.valid_dl)

(70, 3)

In [29]:
13017/70/bs

2.905580357142857

In [30]:
it = iter(data.valid_dl)
x1, y1 = next(it)
x2, y2 = next(it)
x3, y3 = next(it)
it.close()

In [35]:
x1.shape, x2.shape, x3.shape

(torch.Size([64, 70]), torch.Size([64, 70]), torch.Size([64, 70]))

In [34]:
x1.numel() + x2.numel() + x3.numel()

13440

In [36]:
3*64*70

13440

In [37]:
13017/70

185.95714285714286

In [38]:
x1[:,0]

tensor([ 2,  9, 11, 12, 13, 11, 10,  9, 10, 14, 19, 25, 19, 15, 16, 11, 19,  9,
        10,  9, 19, 25, 19, 11, 19, 11, 10,  9, 19, 20, 11, 26, 20, 23, 20, 20,
        24, 20, 11, 14, 11, 11,  9, 14,  9, 20, 10, 20, 35, 17, 11, 10,  9, 17,
         9, 20, 10, 20, 11, 20, 11, 20, 20, 20], device='cuda:0')

In [39]:
y1[:,0]

tensor([19, 19, 27, 10,  9, 12, 32, 19, 26, 10, 11, 15, 11, 10,  9, 15, 11, 19,
        26, 19, 11, 18, 11, 18,  9, 18, 21, 19, 10, 10, 20,  9, 11, 16, 11, 11,
        13, 11, 13,  9, 13, 14, 20, 10, 20, 11, 24, 11,  9,  9, 16, 17, 20, 10,
        20, 11, 24, 11, 19,  9, 19, 11, 11, 10], device='cuda:0')

I believe the data is broken by the following, we have 13017 words in the valid. We create snippets of 70 words which gives about ~185 segments we partion the 185 segments into batches of 64 which gives us the batches. The x input is one segment and the y is x shifted by one word. So mapping from 70 element word vector to 70 element word vector.

In [40]:
v = data.valid_ds.vocab

In [41]:
v.textify(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight'

In [42]:
v.textify(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand'

In [43]:
v.textify(x2[0])

'thousand eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two ,'

In [44]:
v.textify(y2[0])

'eighteen , eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight'

In [45]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


## Single Fully Connected Model

In [46]:
data = src.databunch(bs=bs, bptt=3)

In [47]:
x, y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

model to predict the next words given the previous three 

In [48]:
nv = len(v.itos); nv

40

In [51]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [55]:
nh = 64

here the input to cross entropy is vec compare with index of class thus target[:,-1], input is a batch!

In [57]:
def loss4(input, target):
    return F.cross_entropy(input, target[:,-1])

def acc4(input, target):
    return accuracy(input, target[:,-1])

In [58]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        # place our vocab into a space of
        # dimension nh=64, our input has nv=40 values
        # which map to the embedding nv x nh
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn  = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        # x to embedding which is matrix multiplication if
        # you want by 1 hot encoding, then to hidden state
        # then hidden state.
        h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
        
        if x.shape[1] > 1:
            # add second word to hidden state and redo
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1] > 2:
            # add third word to hidden state and redo
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [72]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [73]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.564458,3.576833,0.071002,00:00
1,2.989226,3.046034,0.308134,00:00
2,2.406881,2.560596,0.398897,00:00
3,2.103259,2.323891,0.418658,00:00
4,1.986859,2.246178,0.434513,00:00
5,1.962597,2.235725,0.434972,00:00


## Same thing with loop 

In [65]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        # create matrix of zeros shape batch x embedding length
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [68]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [69]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.610312,3.567566,0.152344,00:00
1,3.070369,3.092113,0.385110,00:00
2,2.486505,2.604643,0.416131,00:00
3,2.157182,2.336963,0.410846,00:00
4,2.028056,2.240970,0.400735,00:00
5,2.001419,2.227621,0.399127,00:00


## Multi-Fully Connected Model 

In [74]:
data = src.databunch(bs=bs, bptt=20)

In [75]:
x, y = data.one_batch()
x.shape, y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

compare n inputs to n outputs, hence we stack

In [77]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [79]:
learn = Learner(data, Model2(), metrics=accuracy)

In [82]:
learn.model

Model2(
  (i_h): Embedding(40, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=40, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [83]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.717030,3.684199,0.076491,00:00
1,3.626467,3.568501,0.137642,00:00
2,3.513105,3.474238,0.226634,00:00
3,3.393419,3.390173,0.291548,00:00
4,3.278808,3.320656,0.323935,00:00
5,3.180515,3.267990,0.333381,00:00
6,3.104392,3.232973,0.338281,00:00
7,3.051402,3.214243,0.340767,00:00
8,3.018966,3.207092,0.341548,00:00
9,3.002071,3.206036,0.341619,00:00


In [85]:
tensor_one = torch.tensor([[1,2,3],[4,5,6]])
tensor_one

tensor([[1, 2, 3],
        [4, 5, 6]])

In [87]:
tensor_two = torch.tensor([[7,8,9],[10,11,12]])
tensor_two

tensor([[ 7,  8,  9],
        [10, 11, 12]])

In [88]:
tensor_tre = torch.tensor([[13,14,15],[16,17,18]])
tensor_tre

tensor([[13, 14, 15],
        [16, 17, 18]])

In [90]:
tensor_list = [tensor_one, tensor_two, tensor_tre]
tensor_list, len(tensor_list)

([tensor([[1, 2, 3],
          [4, 5, 6]]), tensor([[ 7,  8,  9],
          [10, 11, 12]]), tensor([[13, 14, 15],
          [16, 17, 18]])], 3)

In [91]:
stacked_tensor = torch.stack(tensor_list)
stacked_tensor

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 7,  8,  9],
         [10, 11, 12]],

        [[13, 14, 15],
         [16, 17, 18]]])

In [93]:
stacked_tensor = torch.stack(tensor_list, dim=1)
stacked_tensor

tensor([[[ 1,  2,  3],
         [ 7,  8,  9],
         [13, 14, 15]],

        [[ 4,  5,  6],
         [10, 11, 12],
         [16, 17, 18]]])

In [94]:
stacked_tensor.shape

torch.Size([2, 3, 3])

I'm not sure how fastai is doing this but I believe the output is being stacked meaning our output consits of 
[[[output word one line 1], [output word one line 2], ..]], [[output word two line 1], [output word two line 2], ..]]]
which becomes after stack
[[[output word one line 1], [output word two line 2], ..]], ...]

is fastai able to recognize that an output of word vectors needs to be compared with y of word vectors?

## Maintain State 

In [95]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [96]:
learn = Learner(data, Model3(), metrics=accuracy)

In [97]:
learn.fit_one_cycle(20, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.635440,3.574348,0.084517,00:00
1,3.409752,3.163113,0.332457,00:00
2,2.892196,2.288353,0.460653,00:00
3,2.246084,1.873813,0.468324,00:00
4,1.835033,1.905241,0.328693,00:00
5,1.619364,1.810465,0.432670,00:00
6,1.465574,1.723857,0.445952,00:00
7,1.327130,1.619750,0.492401,00:00
8,1.214984,1.649081,0.513565,00:00
9,1.114001,1.665357,0.541974,00:00


## RNN 

In [98]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.rnn = nn.RNN(nh, nh, batch_first=True)
        self.h_o = nn.Linear(nh, nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res, h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [99]:
learn = Learner(data, Model4(), metrics=accuracy)

In [100]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.572953,3.490906,0.164276,00:00
1,3.106826,2.582470,0.460724,00:00
2,2.412191,1.944239,0.465412,00:00
3,1.910164,1.959616,0.317685,00:00
4,1.615668,1.751909,0.493750,00:00
5,1.398804,1.803212,0.479261,00:00
6,1.207164,1.611339,0.523508,00:00
7,1.036243,1.561898,0.529616,00:00
8,0.884073,1.507772,0.540909,00:00
9,0.749792,1.445756,0.554830,00:00


## 2-Layer GRU 

In [106]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh, nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res, h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [107]:
learn = Learner(data, Model5(), metrics=accuracy)

In [108]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.044491,2.490412,0.441122,00:00
1,1.884692,1.432829,0.598366,00:00
2,0.948264,1.108195,0.756818,00:00
3,0.450037,0.907966,0.803196,00:00
4,0.220491,0.833200,0.804474,00:00
5,0.112463,0.853970,0.819815,00:00
6,0.060552,0.866705,0.809872,00:00
7,0.034536,0.922589,0.817330,00:00
8,0.021232,0.987112,0.798580,00:00
9,0.014464,0.995736,0.797940,00:00
